In [1]:
from setfit import SetFitModel, Trainer, TrainingArguments, sample_dataset
from datasets import load_dataset

2024-03-15 10:23:50.039554: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Initializing a new SetFit model
model = SetFitModel.from_pretrained("WhereIsAI/UAE-Large-V1", use_differentiable_head=True, head_params={"out_features": 2})

No sentence-transformers model found with name WhereIsAI/UAE-Large-V1. Creating a new one with MEAN pooling.
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [3]:
# Preparing the dataset
dataset = load_dataset("SetFit/sst2")
train_dataset = sample_dataset(dataset["train"], label_column="label", num_samples=32)
test_dataset = dataset["test"]

/Users/jiaying/anaconda/anaconda3/envs/LLMsearch/lib/python3.9/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")
/Users/jiaying/anaconda/anaconda3/envs/LLMsearch/lib/python3.9/site-packages/setfit/data.py:154: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.apply(lambda x: x.sample(min(num_samples, len(x)), random_state=seed))


In [4]:
# Preparing the training arguments
args = TrainingArguments(
    batch_size=(32, 16),
    num_epochs=(3, 8),
    end_to_end=True,
    body_learning_rate=(2e-5, 5e-6),
    head_learning_rate=2e-3,
    l2_weight=0.01,
)

# Preparing the trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
)
trainer.train()

Map:   0%|          | 0/64 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 2112
  Batch size = 32
  Num epochs = 3
  Total optimization steps = 198
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jyli1999. Use `wandb login --relogin` to force relogin
/Users/jiaying/anaconda/anaconda3/envs/LLMsearch/lib/python3.9/site-packages/wandb/sdk/lib/ipython.py:77: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display  # type: ignore


  0%|          | 0/198 [00:00<?, ?it/s]

  0%|          | 0/198 [00:00<?, ?it/s]

{'embedding_loss': 0.2262, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.02}
{'embedding_loss': 0.0001, 'learning_rate': 1.662921348314607e-05, 'epoch': 0.76}
{'embedding_loss': 0.0001, 'learning_rate': 1.101123595505618e-05, 'epoch': 1.52}
{'embedding_loss': 0.0001, 'learning_rate': 5.393258426966292e-06, 'epoch': 2.27}


...

Epoch:   0%|          | 0/8 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
from sentence_transformers import util
sentences1 = ['The cat sits outside',
             'A man reads books',
             'The new movie is awesome']

sentences2 = ['The dog plays in the garden',
              'A woman watches TV',
              'The new movie is so great']

# Compute embedding for both lists
embeddings1 = model.model_body.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.model_body.encode(sentences2, convert_to_tensor=True)

# Compute cosine-similarities
cosine_scores = util.cos_sim(embeddings1, embeddings2)

In [ ]:
cosine_scores

tensor([[0.7838, 0.6784, 0.6602],
        [0.5866, 0.7022, 0.6601],
        [0.6920, 0.7290, 0.9955]])